# 🔭 Spectral Fitting Analysis (Google Colab)

Notebook สำหรับวิเคราะห์และ fit สเปกตรัมรังสีเอกซ์ของ AGN
รองรับการกำหนดช่วง Energy Range และเลือก Model ได้เอง

### 📝 วิธีใช้:
1. **Upload** โฟลเดอร์ `Colab_Spectral_Fitting` ทั้งหมดไปที่ Google Drive
2. **Mount Drive** (รัน Cell แรก)
3. **กำหนดค่า** Energy Range และ File Paths ใน Section 1
4. **Load Data** ใน Section 2 (อ่านข้อมูลเข้ามาก่อน)
5. **ทดลอง Model** ใน Section 3 (ทีละโมเดล)
6. **Final Fit** ใน Section 4 (รันสรุปผล)

---

## 0. Setup & Mount Drive

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Change directory to where you uploaded the folder
import os
try:
    os.chdir('/content/drive/MyDrive/Colab_Spectral_Fitting')
    print("✅ Converted Correctly")
except:
    print("⚠️ Warning: Check your path /content/drive/MyDrive/...")

# Verify files
!ls -F

## 1. Configuration - กำหนดค่าก่อนรัน

In [ ]:
# ==============================
# 🎯 ENERGY RANGE (keV)
# กำหนดช่วงพลังงานที่จะวิเคราะห์
# ==============================
ENERGY_MIN = 0.3    # keV (ขอบล่าง)
ENERGY_MAX = 10.0   # keV (ขอบบน)

# ==============================
# 📁 FILE PATHS
# กำหนด path ไปยังไฟล์ข้อมูล
# ==============================
SOURCE_SPECTRUM_PATH = '/content/drive/MyDrive/data/source.pha'    # 📊 Source Spectrum
BACKGROUND_PATH = '/content/drive/MyDrive/data/background.pha'     # 🌌 Background Spectrum
ARF_PATH = '/content/drive/MyDrive/data/source.arf'                # 📈 ARF File
RMF_PATH = '/content/drive/MyDrive/data/source.rmf'                # 🔲 RMF File

# ==============================
# 🔧 FITTING PARAMETERS
# ==============================
N_STEPS = 10           # จำนวน steps สำหรับ grid search (5-20)
USE_BRUTE_FORCE = False # แนะนำ False (Optimization) สำหรับการลองเล่น

### 1.1 🔍 Check File Header (Optional)

In [ ]:
FILE_TO_INSPECT = SOURCE_SPECTRUM_PATH

from astropy.io import fits
import os

if os.path.exists(FILE_TO_INSPECT):
    print(f"📄 Inspecting: {os.path.basename(FILE_TO_INSPECT)}")
    print("=" * 60)
    try:
        with fits.open(FILE_TO_INSPECT) as hdul:
            hdul.info()
            ext_idx = 1 if len(hdul) > 1 else 0
            header = hdul[ext_idx].header
            print(f"\n📋 Key Headers:")
            for key in ['TELESCOP', 'INSTRUME', 'EXPOSURE', 'BACKSCAL']:
                if key in header: print(f"{key:<10} = {header[key]}")
    except Exception as e:
        print(f"❌ Error: {e}")
else:
    print("❌ File not found.")

## 2. Load & Prepare Data (Moved Up)
โหลดข้อมูลก่อน เพื่อให้พร้อมสำหรับการทดสอบ Model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import sys
import importlib
import copy

# Import & Reload
import data_loader
import Models
importlib.reload(data_loader)
importlib.reload(Models)

from data_loader import (
    read_spectrum_file, 
    read_arf_file, 
    read_rmf_file,
    subtract_background,
    get_energy_from_response,
    filter_energy_range,
    fold_model_through_response
)
from Models import combined_model

print("📊 Loading Data...")

# 1. Load Files
source_spec = read_spectrum_file(SOURCE_SPECTRUM_PATH)
bkg_spec = read_spectrum_file(BACKGROUND_PATH)
arf_data = read_arf_file(ARF_PATH)
rmf_data = read_rmf_file(RMF_PATH)

# 2. Prepare Data
energy = get_energy_from_response(arf_data, rmf_data)

if source_spec and bkg_spec:
    net_counts, net_error = subtract_background(source_spec, bkg_spec)
else:
    net_counts = source_spec.counts
    net_error = np.sqrt(np.maximum(net_counts, 1.0))

exposure = source_spec.exposure if source_spec else 1.0
observed_rate = net_counts / exposure
observed_error = net_error / exposure

# Filter Energy Range
energy_filtered, rate_filtered, error_filtered = filter_energy_range(
    energy, observed_rate, observed_error, 
    e_min=ENERGY_MIN, e_max=ENERGY_MAX
)

# Filter ARF
arf_filtered = None
if arf_data is not None:
    arf_filtered = copy.deepcopy(arf_data)
    min_len = min(len(energy), len(observed_rate))
    energy_aligned = energy[:min_len]
    mask = (energy_aligned >= ENERGY_MIN) & (energy_aligned <= ENERGY_MAX)
    
    # Align ARF
    if len(arf_filtered.arf) > len(mask):
         arf_filtered.arf = arf_filtered.arf[:len(mask)]
         if arf_filtered.energy_lo is not None: arf_filtered.energy_lo = arf_filtered.energy_lo[:len(mask)]
         if arf_filtered.energy_hi is not None: arf_filtered.energy_hi = arf_filtered.energy_hi[:len(mask)]
         if arf_filtered.energy_mid is not None: arf_filtered.energy_mid = arf_filtered.energy_mid[:len(mask)]

    arf_filtered.arf = arf_filtered.arf[mask]
    # Ensure midpoints update if needed for future logic, though mostly just arf array matters
    if arf_filtered.energy_mid is not None: arf_filtered.energy_mid = arf_filtered.energy_mid[mask]
    
print(f"✅ Data Ready: {len(energy_filtered)} bins ({ENERGY_MIN}-{ENERGY_MAX} keV)")

## 3. 🧪 Step-by-Step Model Exploration
ทดลองใส่ทีละ Model แล้วพล็อตกราฟดูผลลัพธ์ (ตามลำดับที่ต้องการ)

In [ ]:
# ----- Helper Function for Quick Plot (รัน Cell นี้ก่อน) -----
def quick_fit_and_plot(model_list, title="Model Test"):
    print(f"\n🧪 Testing Model: {model_list}")
    
    # 1. Setup Defaults
    current_params = {
        'pl_norm': 1e-3, 'photon_index': 1.7,
        'nH': 0.1,
        'bb_norm': 1e-4, 'kT': 0.1,
        'line_norm': 1e-4, 'line_energy': 6.4, 'line_sigma': 0.1,
        'refl_norm': 1e-4
    }
    
    # 2. Define Chi-Squared (Local)
    def local_chi2(param_vals, param_keys):
        p = current_params.copy()
        for k, v in zip(param_keys, param_vals):
            p[k] = v
        
        model_flux = combined_model(energy_filtered, p, model_list)
        if arf_filtered:
            model_rate = fold_model_through_response(model_flux, arf_filtered)
        else:
            model_rate = model_flux
            
        mask = error_filtered > 0
        return np.sum(((rate_filtered[mask] - model_rate[mask]) / error_filtered[mask])**2)

    # 3. Optimize
    # Identify active params based on models
    active_keys = []
    if 'powerlaw' in model_list: active_keys.extend(['pl_norm', 'photon_index'])
    if 'tbabs' in model_list: active_keys.extend(['nH'])
    if 'blackbody' in model_list: active_keys.extend(['bb_norm', 'kT'])
    if 'gaussian' in model_list: active_keys.extend(['line_norm', 'line_energy'])
    if 'reflection' in model_list: active_keys.extend(['refl_norm'])
    
    initial_vals = [current_params[k] for k in active_keys]
    bounds = [(1e-6, 1e2)] * len(active_keys) # Simple bounds
    
    res = minimize(local_chi2, initial_vals, args=(active_keys,), bounds=bounds, method='L-BFGS-B')
    best_p = current_params.copy()
    for k, v in zip(active_keys, res.x):
        best_p[k] = v
        
    print(f"✅ Best Chi-sq/dof: {res.fun/(len(rate_filtered)-len(active_keys)):.2f}")
    
    # 4. Plot
    flux = combined_model(energy_filtered, best_p, model_list)
    rate = fold_model_through_response(flux, arf_filtered) if arf_filtered else flux
    
    plt.figure(figsize=(8, 5))
    plt.errorbar(energy_filtered, rate_filtered, yerr=error_filtered, fmt='.', color='gray', alpha=0.5, label='Data')
    plt.plot(energy_filtered, rate, 'r-', lw=2, label=f'Model: {title}')
    plt.yscale('log'); plt.xscale('log')
    plt.xlabel('Energy (keV)'); plt.ylabel('Count Rate')
    plt.legend(); plt.title(f"{title} (Reduced $\chi^2$={res.fun/(len(rate_filtered)-len(active_keys)):.2f})")
    plt.grid(True, alpha=0.3)
    plt.show()

### 3.1 Step 1: Powerlaw Only
ดูแค่ Continuum หลักก่อน

In [ ]:
quick_fit_and_plot(['powerlaw'], title="Powerlaw Only")

### 3.2 Step 2: Add Reflection
เพิ่มส่วนสะท้อนกลับ (Reflection Hump)

In [ ]:
quick_fit_and_plot(['powerlaw', 'reflection'], title="Powerlaw + Reflection")

### 3.3 Step 3: Add Absorption (TBabs)
เพิ่มการดูดกลืนแสง (ควรทำให้กราฟช่วงต้นๆ ต่ำลง)

In [ ]:
quick_fit_and_plot(['powerlaw', 'reflection', 'tbabs'], title="PL + Refl + Abs")

### 3.4 Step 4: Add Gaussian Line
เพิ่มเส้น Fe Kalpha (ที่ ~6.4 keV)

In [ ]:
quick_fit_and_plot(['powerlaw', 'reflection', 'tbabs', 'gaussian'], title="... + Gaussian Line")

### 3.5 Step 5: Add Blackbody
เพิ่มส่วน Soft Excess (ช่วงพลังงานต่ำ)

In [ ]:
quick_fit_and_plot(['powerlaw', 'reflection', 'tbabs', 'gaussian', 'blackbody'], title="Full Model")

## 4. Final Full Analysis
รันการวิเคราะห์ละเอียด พร้อมคำนวณ Error และ Residuals แบบเต็มรูปแบบ

In [ ]:
# เลือกโมเดลสุดท้ายที่ต้องการ
SELECTED_MODELS = ['powerlaw', 'tbabs'] 
# SELECTED_MODELS = ['powerlaw', 'reflection', 'tbabs', 'gaussian', 'blackbody']

# Initial Params & Ranges (สำหรับ Final Fit)
INITIAL_PARAMS = {
    'pl_norm': 0.01, 'photon_index': 2.0,
    'nH': 0.1,
    'bb_norm': 0.1, 'kT': 0.5,
    'line_energy': 6.4, 'line_sigma': 0.1, 'line_norm': 1.0,
    'refl_norm': 0.5,
}

PARAM_RANGES = {
    'pl_norm': (1e-4, 1.0), 'photon_index': (1.5, 2.5), 'nH': (0.01, 1.0),
    'refl_norm': (1e-4, 1.0), 'line_norm': (1e-4, 1.0)
}

# -------- FITTING LOGIC (Compact) --------
def calculate_final_chi2(params_dict, models):
    flux = combined_model(energy_filtered, params_dict, models)
    model_rate = fold_model_through_response(flux, arf_filtered) if arf_filtered else flux
    mask = error_filtered > 0
    return np.sum(((rate_filtered[mask] - model_rate[mask])/error_filtered[mask])**2)

# 1. OptimizationFit
param_names = []
if 'powerlaw' in SELECTED_MODELS: param_names += ['pl_norm', 'photon_index']
if 'tbabs' in SELECTED_MODELS: param_names += ['nH']
if 'blackbody' in SELECTED_MODELS: param_names += ['bb_norm', 'kT']
if 'gaussian' in SELECTED_MODELS: param_names += ['line_norm', 'line_energy']
if 'reflection' in SELECTED_MODELS: param_names += ['refl_norm']

def objective(p_arr):
    p = INITIAL_PARAMS.copy()
    for k, v in zip(param_names, p_arr): p[k] = v
    return calculate_final_chi2(p, SELECTED_MODELS)

print(f"🎯 Final Fitting using: {SELECTED_MODELS}")
res = minimize(objective, [INITIAL_PARAMS[k] for k in param_names], method='L-BFGS-B', bounds=[(1e-5, 100)]*len(param_names))

best_params = INITIAL_PARAMS.copy()
for k, v in zip(param_names, res.x): best_params[k] = v
chi2 = res.fun
dof = len(rate_filtered) - len(param_names)

print(f"✅ Results: Reduced χ² = {chi2/dof:.3f}")
for k, v in best_params.items():
    if k in param_names: print(f"  {k}: {v:.4f}")

# Plotting (Standard)
best_flux = combined_model(energy_filtered, best_params, SELECTED_MODELS)
best_rate = fold_model_through_response(best_flux, arf_filtered) if arf_filtered else best_flux

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), height_ratios=[3, 1], sharex=True)
ax1.errorbar(energy_filtered, rate_filtered, yerr=error_filtered, fmt='o', alpha=0.5, label='Data')
ax1.plot(energy_filtered, best_rate, 'r-', lw=2, label='Best Fit')
ax1.set_yscale('log'); ax1.set_xscale('log'); ax1.set_ylabel('Count Rate'); ax1.legend()
res_val = (rate_filtered - best_rate) / error_filtered
ax2.axhline(0, color='gray', linestyle='--')
ax2.plot(energy_filtered, res_val, 'ko', markersize=2)
ax2.set_ylabel('Residuals (σ)'); ax2.set_xlabel('Energy (keV)'); ax2.set_ylim(-5, 5)
plt.tight_layout(); plt.show()